In [2]:
!pip install cohere umap-learn altair annoy datasets tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 47.1 MB/s eta 0:00:0000:01
  Attempting uninstall: fastavro
    Found existing installation: fastavro 1.7.4
    Uninstalling fastavro-1.7.4:
      Successfully uninstalled fastavro-1.7.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.


In [3]:

import cohere
import numpy as np
import re
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

/opt/conda/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/conda/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/conda/lib/python3.10/site-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopython' keyword argume

In [9]:
# Get dataset
dataset = load_dataset("trec", split="train")
# Import into a pandas dataframe, take only the first 1000 rows
df = pd.DataFrame(dataset)[:1000]
# Preview the data to ensure it has loaded correctly
df.head(10)

,label-coarse,label-fine,text
0,0,0,How did serfdom develop in and then leave Russia ?
1,1,1,What films featured the character Popeye Doyle ?
2,0,0,How can I find a list of celebrities ' real names ?
3,1,2,What fowl grabs the spotlight after the Chinese Year of the Monkey ?
4,2,3,What is the full form of .com ?
5,3,4,What contemptible scoundrel stole the cork from my lunch ?
6,3,5,What team did baseball 's St. Louis Browns become ?
7,3,6,What is the oldest profession ?
8,0,7,What are liver enzymes ?
9,3,4,Name the scar-faced bounty hunter of The Old West .


In [14]:
import pandas as pd
import pyarrow.parquet as pq
# Replace 'your_file.parquet' with the path to your Parquet file
# parquet_file_path = '/kaggle/input/parquet-abhay-dataset/0000-3.parquet'

# # Use pyarrow to read the Parquet file into a Pandas DataFrame
# df = pq.read_table(parquet_file_path).to_pandas()
df=pd.read_csv('/kaggle/input/working-smriti-book-sementic-search/smiriti_book_sementic_search.csv')

In [15]:
df

,Unnamed: 0,text
0,0,"How to Fall Asleep Fast\nand Sleep Better\nBy Melinda Smith, M.A. and Lawrence Robinson"
1,1,"\n\nImproving sleep hygiene to get a better\nnight's sleep\nSleeping well directly affects your mental and physical health. Fall short and it can take a\n\nserious toll on your daytime energy, productivity, emotional balance, and even your\n\nweight. Yet many of us regularly toss and turn at night, struggling to get the sleep we\n need."
2,2,"Getting a good night’s sleep may seem like an impossible goal when you’re wide awake at\n\n3 a.m., but you have much more control over the quality of your sleep than you probably\n\nrealize. Just as the way you feel during your waking hours often hinges on how well you\n\nsleep at night, so the cure for sleep difficulties can often be found in your daily routine."
3,3,"Unhealthy daytime habits and lifestyle choices can leave you tossing and turning at night\n\nand adversely affect your mood, brain and heart health, immune system, creativity,\n\nvitality, and weight. Whether you struggle to fall asleep when you get into bed or wake up\n\nin the night and find it difficult to go back to sleep, the solution often lies in establishing\n\nhealthy sleep habits, known as “sleep hygiene”. By improving your sleep hygiene, you can\n\nfall asleep faster, stay asleep for longer, boost your overall health, and improve how you\n\nthink and feel during the day."
4,4,"\n\nHow fast should I fall asleep?\nEveryone is different, but most healthy adults should be able to fall asleep at night within\n10 to 20 minutes. If it takes you longer than 30 minutes, it may be a sign that you have an\n\nunderlying condition, such as insomnia."
5,5,"However, it’s also possible to fall asleep too quickly. If you regularly fall asleep as soon as\n\nyour head touches the pillow, it may be a sign that you’re suffering from sleep deprivation.\n\nIn other words, you’re not getting enough quality sleep at night."
6,6,"This could be due to a sleep disorder, such as sleep apnea, where your breathing is\n\nfrequently interrupted while you sleep, disrupting your rest at night and leaving you feeling\n\ntired during the day. Sleep apnea is often accompanied by loud snoring, choking, or\n\ngasping while you sleep. It can cause serious health problems so it’s important to speak to\n\nyour doctor right away."
7,7,"Whether you’re falling asleep too fast or if it’s taking you an age to finally drop off, by\n\nexperimenting with the following tips, you can improve your sleep hygiene and enjoy\n better, more refreshing sleep at night."
8,8,"\n\nTip 1: Keep in sync with your body's natural\nsleep-wake cycle\nGetting in sync with your body's natural sleep-wake cycle, or circadian rhythm, is one of\n\nthe most important strategies for sleeping better. If you keep a regular sleep-wake\n\nschedule, you'll feel much more refreshed and energized than if you sleep the same\n\nnumber of hours at different times, even if you only alter your sleep schedule by an hour\n\nor two."
9,9,"Try to go to sleep and get up at the same time every day. This helps set your body's\n\ninternal clock and optimize the quality of your sleep. Choose a bed time when you\n\nnormally feel tired, so that you don't toss and turn. If you're getting enough sleep, you\n\nshould wake up naturally without an alarm. If you need an alarm clock, you may need an\n\nearlier bedtime."


In [16]:
# Paste your API key here. Remember to not share publicly
api_key = '(for_privacy_it_is_removed)'
#AXj3kBMFUPTtPSSEHWt6RwaPSgjVdjCJmyv7gF6f
# Create and retrieve a Cohere API key from dashboard.cohere.ai/welcome/register
co = cohere.Client(api_key)

# Get the embeddings
embeds = co.embed(texts=list(df['text']),
                  model='embed-english-v2.0').embeddings

In [17]:
# Create the search index, pass the size of embedding
search_index = AnnoyIndex(np.array(embeds).shape[1], 'angular')
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])
search_index.build(10) # 10 trees
search_index.save('test.ann')

True

In [18]:
questioin='how fast i should sleep'

In [19]:
query = f"Question:{questioin}\nNearest neighbors:"

# Get the query's embedding
query_embed = co.embed(texts=[query],
                  model="embed-english-v2.0").embeddings

# Retrieve the nearest neighbors
similar_item_ids = search_index.get_nns_by_vector(query_embed[0],10,
                                                include_distances=True)
# Format the results
results = pd.DataFrame(data={'texts': df.iloc[similar_item_ids[0]]['text'], 
                             'distance': similar_item_ids[1]})


print(f"Query:'{query}'\nNearest neighbors:")
results

Query:'Question:how fast i should sleep
Nearest neighbors:'
Nearest neighbors:


,texts,distance
36,\n\nTip 5: Improve your sleep environment\nA peaceful bedtime routine sends a powerful signal to your brain that it's time to wind\n\ndown and let go of the day's stresses. Sometimes even small changes to your\n\nenvironment can make a big difference to how fast you fall asleep and the quality of your\n\nsleep.,0.923042
7,"Whether you’re falling asleep too fast or if it’s taking you an age to finally drop off, by\n\nexperimenting with the following tips, you can improve your sleep hygiene and enjoy\n better, more refreshing sleep at night.",0.954270
53,"\n\nTip 8: Learn how to get back to sleep if you\nwake up\nIt's normal to wake briefly during the night but if you're having trouble falling back asleep,\n\nthese tips may help:",0.957523
9,"Try to go to sleep and get up at the same time every day. This helps set your body's\n\ninternal clock and optimize the quality of your sleep. Choose a bed time when you\n\nnormally feel tired, so that you don't toss and turn. If you're getting enough sleep, you\n\nshould wake up naturally without an alarm. If you need an alarm clock, you may need an\n\nearlier bedtime.",0.969375
22,"When it's time to sleep, make sure the room is dark. Use heavy curtains or shades to\n\nblock light from windows, or try a sleep mask. Also consider covering up electronics that\n\nemit light.",0.978955
28,"\n\nTip 4: Be smart about what you eat and\ndrink\nYour daytime eating habits play a role in how well you sleep, especially in the hours before\n\nbedtime.",0.980045
38,Keep your room cool. Most people sleep best in a slightly cool room (around 65° F or 18°\n\nC) with adequate ventilation. A bedroom that is too hot or too cold can interfere with\n quality sleep.,0.983048
57,"Do a quiet, non-stimulating activity. If you've been awake for more than 15 minutes, get\n\nout of bed and do a quiet, non-stimulating activity, such as reading a book. Keep the lights\n\ndim and avoid screens so as not to cue your body that it's time to wake up.",0.986472
24,"\n\nTip 3: Exercise during the day\nPeople who exercise regularly fall asleep faster at night, sleep better, and feel less sleepy\n\nduring the day. Regular exercise also improves the symptoms of insomnia and sleep apnea\n\nand increases the amount of time you spend in the deep, restorative stages of sleep.",0.988044
27,"Try to finish moderate to vigorous workouts at least three hours before bedtime. If you're\n\nstill experiencing sleep difficulties, move your workouts even earlier. Relaxing, low-impact\n\nexercises such as yoga or gentle stretching in the evening can help promote sleep.",0.998577
